In [2]:
import pandas as pd
import re
import shutil
import bisect

In [3]:
buildings_file = "input/export_descr_buildings.txt"
backup_buildings_file = "input/export_descr_buildings_backup.txt"
temp_buildings_file = "input/export_descr_buildings_temp.txt"
shutil.copy(buildings_file, backup_buildings_file)

'input/export_descr_buildings_backup.txt'

# Unit cross check fixes

In [4]:
unit_cross_check_issues = pd.read_csv("output/unit_cross_check_issues.csv")
unit_cross_check_issues.set_index("type", inplace=True)
unit_cross_check_issues

,in_units,aor_units,merc_units,in_buildings,aor_buildings,merc_buildings
type,,,,,,
barb peasant slave,1,0,0,0,0,0
italic velites,1,1,0,0,1,0
italic hastati,1,1,0,0,1,0
italic principes,1,1,0,0,1,0
italic triarii,1,1,0,0,0,0
...,...,...,...,...,...,...
legio xxii primigenia,1,0,0,0,0,0
legio xxii primigenia first,1,0,0,0,0,0
roman early praetorian,1,0,0,0,0,0


In [5]:
all_units = pd.read_csv("output/ris_units_combined.csv")
all_units.set_index("type", inplace=True)

In [6]:
factions = all_units["faction"].dropna().drop_duplicates()
factions = factions.str.split(", ")
factions = factions.apply(pd.Series).stack().reset_index(drop=True)
factions = factions.drop_duplicates()
factions = factions.str.strip()
factions = set(factions.sort_values().to_list())
factions

{'acarnania',
 'achaea',
 'acragas',
 'aedui',
 'aetolia',
 'all',
 'allobroges',
 'anatolian',
 'anatolians',
 'antigonid',
 'arab',
 'ardiaei',
 'arevaci',
 'argos',
 'armenia',
 'arverni',
 'asti',
 'athens',
 'atropatene',
 'bactria',
 'belgae',
 'bessi',
 'bithynia',
 'boeotia',
 'boii',
 'bosporan',
 'byzantium',
 'cappadocia',
 'carthage',
 'carthaginian',
 'chatti',
 'chersonesus',
 'chios',
 'chrysaoria',
 'cilicians',
 'cimbri',
 'cius',
 'cyrene',
 'cyzicus',
 'dentheletae',
 'e_hellenistic',
 'eastern',
 'edetani',
 'egypt',
 'elis',
 'emporion',
 'epirus',
 'ethiopian',
 'galatians',
 'getae',
 'gortyn',
 'greek',
 'greeks',
 'histria',
 'iberian',
 'indian',
 'insubres',
 'iranian',
 'issa',
 'kabyle',
 'knossos',
 'kush',
 'kydonia',
 'libyan',
 'lugii',
 'lusitani',
 'lycia',
 'lysiad',
 'lyttos',
 'maedi',
 'masaesyli',
 'massalia',
 'massylii',
 'mauryan',
 'megalopolis',
 'messene',
 'miletus',
 'nabataea',
 'odrysians',
 'olbia',
 'paionia',
 'paphlagonia',
 'parni'

In [7]:
test_unit = "aor italic velites"
test_unit_factions = set(all_units.loc[test_unit, "faction"].split(", "))
cultures = factions - test_unit_factions
cultures

{'all',
 'anatolian',
 'arab',
 'carthaginian',
 'e_hellenistic',
 'eastern',
 'ethiopian',
 'greek',
 'iberian',
 'indian',
 'iranian',
 'libyan',
 'roman,',
 'w_hellenistic'}

In [8]:
all_factions = factions - cultures
all_factions

{'acarnania',
 'achaea',
 'acragas',
 'aedui',
 'aetolia',
 'allobroges',
 'anatolians',
 'antigonid',
 'ardiaei',
 'arevaci',
 'argos',
 'armenia',
 'arverni',
 'asti',
 'athens',
 'atropatene',
 'bactria',
 'belgae',
 'bessi',
 'bithynia',
 'boeotia',
 'boii',
 'bosporan',
 'byzantium',
 'cappadocia',
 'carthage',
 'chatti',
 'chersonesus',
 'chios',
 'chrysaoria',
 'cilicians',
 'cimbri',
 'cius',
 'cyrene',
 'cyzicus',
 'dentheletae',
 'edetani',
 'egypt',
 'elis',
 'emporion',
 'epirus',
 'galatians',
 'getae',
 'gortyn',
 'greeks',
 'histria',
 'insubres',
 'issa',
 'kabyle',
 'knossos',
 'kush',
 'kydonia',
 'lugii',
 'lusitani',
 'lycia',
 'lysiad',
 'lyttos',
 'maedi',
 'masaesyli',
 'massalia',
 'massylii',
 'mauryan',
 'megalopolis',
 'messene',
 'miletus',
 'nabataea',
 'odrysians',
 'olbia',
 'paionia',
 'paphlagonia',
 'parni',
 'pentapolis',
 'pergamon',
 'pontus',
 'priene',
 'ptolemaic',
 'rhodes',
 'roman',
 'saba',
 'saka',
 'scordisci',
 'seleucid',
 'selge',
 'sino

In [9]:
resources = all_units["hidden_resources"].dropna().drop_duplicates()
resources = resources.str.split(", ")
resources = resources.apply(pd.Series).stack().reset_index(drop=True)
resources = resources.drop_duplicates()
resources = resources.str.strip()
resources = set(resources.sort_values().to_list())
all_resources = resources
all_resources

{'achaian',
 'aeduia',
 'aestian',
 'aitolian',
 'akarnanian',
 'alanic',
 'albanian',
 'allobrogia',
 'ambrakiote',
 'ankonitan',
 'arab',
 'argive',
 'armenian',
 'arvernia',
 'asascenian',
 'asian',
 'athamanian',
 'athenian',
 'babylonian',
 'bactrian',
 'balearic',
 'bastarnian',
 'belgica',
 'bessian',
 'bithynian',
 'boiaea',
 'boiotian',
 'bosporan',
 'brittonic',
 'byzantine',
 'caledonian',
 'cantabrian',
 'cappadocian',
 'caucasian',
 'chattian',
 'chaucian',
 'chersonesus',
 'cilician',
 'cimbrian',
 'crete',
 'dahaea',
 'deuteroi',
 'egyptian_native',
 'elean',
 'emporionite',
 'epirote',
 'etennian',
 'ethiopian',
 'galatian',
 'gallic',
 'gautic',
 'germanic',
 'getic',
 'gortynian',
 'greek',
 'harian',
 'herakleiote',
 'hyrkania',
 'iberian',
 'illyrian',
 'indian',
 'indo_greek',
 'insubria',
 'ionian',
 'isaurian',
 'issaian',
 'istrian',
 'italic',
 'italiote',
 'judaean',
 'kabylian',
 'karian',
 'kian',
 'knossian',
 'kyrenaian',
 'kyzikan',
 'libyan',
 'lycian',


In [10]:
unit_cross_check_issues = unit_cross_check_issues.merge(all_units[["ownership", "faction"]], how="left", left_index=True, right_index=True)
unit_cross_check_issues.head()

,in_units,aor_units,merc_units,in_buildings,aor_buildings,merc_buildings,ownership,faction
type,,,,,,,,
barb peasant slave,1,0,0,0,0,0,"acarnania, achaea, acragas, aedui, aetolia, al...",NaN
italic velites,1,1,0,0,1,0,slave,NaN
italic hastati,1,1,0,0,1,0,slave,NaN
italic principes,1,1,0,0,1,0,slave,NaN
italic triarii,1,1,0,0,0,0,slave,NaN


## Fix: AoR units exist but aren't assigned buildings

In [11]:
def duplicate_unit_and_update(source_unit, 
                              target_unit, 
                              source_resource=None,
                              target_resource=None,
                              faction_exclude=[],
                              alt_source_unit=None,
                              alt_source_resource=None,
                              building_level=None,
                              missing_factions=set()):
    source_pattern = re.compile(f"^\s+recruit \"{source_unit}\" 0 requires")
    target_pattern = re.compile(f"^\s+recruit \"{target_unit}\" 0 requires")
    alt_pattern = re.compile(f"^\s+recruit \"{alt_source_unit}\" 0 requires")
    primary_match = False

    with open(buildings_file, "r") as file_r, open(temp_buildings_file, "w") as file_w:
        prev_line = file_r.readline()
        line = file_r.readline()

        while prev_line:
            # Check the target unit line doesn't already exist (if it does we don't want to add it again)
            if not target_pattern.match(line):
                # Check for matches with the alternate where the previous line didn't match
                # the primary. This is an indication that it is a missing faction that 
                # needs to use the alternate
                secondary_match = alt_pattern.match(prev_line)
                if secondary_match and not primary_match:
                    try:
                        faction = re.findall("factions \{([a-z2, _]+)\}", prev_line)[0].replace(",","").strip()
                    except:
                        print(prev_line)
                    if faction in missing_factions and faction not in faction_exclude:
                        new_line = prev_line.replace(alt_source_unit, target_unit)           
                        if alt_source_resource is not None:
                            new_line = new_line.replace(alt_source_resource, target_resource)
                        if building_level is not None:
                            new_line = re.sub("_[1-4]", f"_{building_level}", new_line)
                        
                        file_w.write(new_line)
                
                # Write the original line
                file_w.write(prev_line)

                primary_match = source_pattern.match(prev_line)
                if primary_match:
                    try:
                        faction = re.findall("factions \{([a-z2, _]+)\}", prev_line)[0].strip()
                    except:
                        print(prev_line)
                    if faction not in faction_exclude:
                        new_line = prev_line.replace(source_unit, target_unit)           
                        if source_resource is not None:
                            new_line = new_line.replace(source_resource, target_resource)
                        if building_level is not None:
                            new_line = re.sub("_[1-4]", f"_{building_level}", new_line)                            
                        file_w.write(new_line)
            else:
                file_w.write(prev_line)
            
            prev_line = line    
            line = file_r.readline()
        file_w.write(line)

    shutil.move(temp_buildings_file, buildings_file)

In [12]:
unit_cross_check_issues[(unit_cross_check_issues["aor_units"] == 1) 
                        & (unit_cross_check_issues["aor_buildings"] == 0)]

,in_units,aor_units,merc_units,in_buildings,aor_buildings,merc_buildings,ownership,faction
type,,,,,,,,
italic triarii,1,1,0,0,0,0,slave,NaN
towered forest elephants,1,1,1,1,0,0,"carthage, egypt, masaesyli, massylii, slave","masaesyli, massylii"
caetrati,1,1,0,1,0,0,"arevaci, edetani, lusitani, slave","arevaci, edetani, lusitani"
iberian heavy swordsmen,1,1,1,1,0,0,"arevaci, carthage, edetani, lusitani, slave","arevaci, edetani, lusitani"
iberian chosen infantry,1,1,0,1,0,0,"arevaci, edetani, lusitani, slave","arevaci, edetani, lusitani"
iberian heavy cavalry,1,1,1,1,0,0,"arevaci, edetani, lusitani, slave","arevaci, edetani, lusitani"
bellovacian infantry,1,1,1,1,0,0,"aedui, allobroges, arverni, boii, cimbri, gala...","aedui, allobroges, arverni, boii, cimbri, gala..."
scordiscian infantry,1,1,1,1,0,0,"aedui, allobroges, arverni, belgae, boii, cimb...","aedui, allobroges, arverni, belgae, boii, cimb..."
treverian noble cavalry,1,1,1,1,0,0,"aedui, allobroges, arverni, belgae, boii, cimb...","aedui, allobroges, arverni, belgae, boii, cimb..."


List I'm not sure about so haven't altered:
- towered forest elephants (which hidden resource should they require?)
- iberian heavy swordsmen (should they be carthage units or not?)
- bellovacian infantry (which hidden resource should they require?)
- treverian noble cavalry (which hidden resource should they require?)
- galato-thracian warband (which hidden resource should they require?)

In [40]:
def get_details_for_aor_insertion(base_unit, target_building=None, target_level=None):
    """Note that if used on consecutive units (e.g. galatian slingers and galatian skirmishers)
       it'll put them in the wrong order
    """
    target_unit = "aor " + base_unit

    print(f"target_unit: {target_unit}")
    if target_building is None:
        target_building = all_units.loc[base_unit, "building"]
        if type(target_building) != str:
            target_building = input("Please enter the required building (e.g. missile_1):")
    print(f"target_building: {target_building}")

    if target_level is None:
        target_level = all_units.loc[base_unit, "recruitment_building"]
        if not target_level in ("1", "2", "3", "4") :
            target_level = input("Please enter the required recruitment building level (1-4):")
    print(f"target_level: {target_level}")

    # Try to get the target resource by assuming the first word of the base unit is the region (because it often is)
    potential_resource = base_unit.split()[0]
    if potential_resource in all_resources:
        target_resource = potential_resource
    # Sometimes there's slight variation (e.g. insubrian vs insubria) so strip off the last letter
    elif potential_resource[:-1] in all_resources:
        target_resource = potential_resource
    # Or try adding an "n" to the end
    elif potential_resource+"n" in all_resources:
        target_resource = potential_resource+"n"
    # Try to get the target resource by looking for other aor units that have the same region
    else:
        try:
            target_resource = all_units.loc[all_units.index.str.startswith("aor " + potential_resource), "hidden_resources"].dropna().value_counts().index[0]
        except:
            target_resource = input("Please enter the required hidden resource: ")
    print(f"target_resource: {target_resource}") 

    same_level_units = all_units[(all_units["building"] == target_building) 
                                & (all_units["recruitment_building"] == target_level)
                                & (all_units["aor"] == True)]
    same_level_units = same_level_units.sort_index()

    position = bisect.bisect_left(same_level_units.index, target_unit)
    source_unit = same_level_units.index[position-1]
    source_resource = same_level_units.loc[source_unit, "hidden_resources"]
    source_unit_factions = set(all_units.loc[source_unit, "faction"].split(", "))
    missing_factions = all_factions - source_unit_factions

    print(f"source_unit: {source_unit}, source_resource: {source_resource}")
    print(f"missing_factions: {missing_factions}")

    try:
        alt_source_unit = same_level_units.index[position]
        alt_source_resource = same_level_units.loc[alt_source_unit, "hidden_resources"]
        print(f"alt_source_unit: {alt_source_unit}, alt_source_resource: {alt_source_resource}")
    except:
        alt_source_unit = None
        alt_source_resource = None
        print("No alt_source_unit found (probably due to change of level) - check files manually for next unit")

    faction_exclude = unit_cross_check_issues.loc[base_unit, "ownership"].split()
    print(f"faction_exclude: {faction_exclude}")

    return target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, alt_source_resource, faction_exclude

In [31]:
# Requires barracks_3 and level 3
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions,\
    alt_source_unit, alt_source_resource, faction_exclude = get_details_for_aor_insertion("italic triarii", target_building="barracks_3", target_level="3")
faction_exclude.append("roman")
faction_exclude

target_unit: aor italic triarii
target_building: barracks_3
target_level: 3
target_resource: italic
source_unit: aor italic principes, source_resource: italic
missing_factions: set()
alt_source_unit: aor karian heavy infantry, alt_source_resource: karian
faction_exclude: ['slave']


['slave', 'roman']

In [ ]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [17]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, \
    alt_source_unit, alt_source_resource, faction_exclude = get_details_for_aor_insertion("caetrati")

target_unit: aor caetrati
target_building: barracks_1
target_level: 1
target_resource: iberian
source_unit: aor brittonic spearmen, source_resource: brittonic
missing_factions: {'trinovantes'}
alt_source_unit: aor caucasian hillmen, alt_source_resource: caucasian
faction_exclude: ['arevaci,', 'edetani,', 'lusitani,', 'slave']


In [ ]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [18]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("iberian chosen infantry")

target_unit: aor iberian chosen infantry
target_building: barracks_3
target_level: 3
target_resource: iberian
source_unit: aor helvetian infantry, source_resource: allobrogia
missing_factions: {'allobroges'}
alt_source_unit: aor insubrian infantry, alt_source_resource: insubria
faction_exclude: ['arevaci,', 'edetani,', 'lusitani,', 'slave']


In [ ]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [32]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("iberian heavy cavalry")

target_unit: aor iberian heavy cavalry
target_building: equestrian_2
target_level: 2
target_resource: iberian
source_unit: aor hyrkanian horsemen, source_resource: hyrkania
missing_factions: {'parni'}
alt_source_unit: aor illyrian cavalry, alt_source_resource: illyrian
faction_exclude: ['arevaci,', 'edetani,', 'lusitani,', 'slave']


In [12]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [33]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("scordiscian infantry")

target_unit: aor scordiscian infantry
target_building: barracks_2
target_level: 2
target_resource: scordiscian
source_unit: aor rhodian hoplites, source_resource: rhodian
missing_factions: {'rhodes'}
alt_source_unit: aor sicel thureophoroi, alt_source_resource: sicel
faction_exclude: ['aedui,', 'allobroges,', 'arverni,', 'belgae,', 'boii,', 'cimbri,', 'galatians,', 'insubres,', 'slave,', 'tylis,', 'volcae']


In [14]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [37]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("volcaean axemen")

alt_source_unit = "aor argive epilektoi"
alt_source_resource = "argive"

target_unit: aor volcaean axemen
target_building: barracks_3
target_level: 3
target_resource: volcaea
source_unit: aor thracian infantry, source_resource: thracian
missing_factions: {'odrysians', 'kabyle', 'thracians', 'paionia', 'asti', 'bithynia', 'triballi', 'maedi', 'bessi', 'dentheletae'}
No alt_source_unit found (probably due to change of level) - check files manually for next unit
faction_exclude: ['slave,', 'volcae']


In [31]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [42]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("galatian slingers")

target_unit: aor galatian slingers
target_building: missiles_1
target_level: 1
target_resource: galatian
source_unit: aor egyptian slingers, source_resource: egyptian_native
missing_factions: {'egypt'}
alt_source_unit: aor getic archers, alt_source_resource: getic
faction_exclude: ['galatians,', 'scordisci,', 'slave,', 'tylis']


In [20]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [43]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("galatian skirmishers")

target_unit: aor galatian skirmishers
target_building: missiles_1
target_level: 1
target_resource: galatian
source_unit: aor egyptian slingers, source_resource: egyptian_native
missing_factions: {'egypt'}
alt_source_unit: aor getic archers, alt_source_resource: getic
faction_exclude: ['galatians,', 'scordisci,', 'slave,', 'tylis']


In [33]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [45]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("illyrian thureophoroi")

target_unit: aor illyrian thureophoroi
target_building: barracks_2
target_level: 2
target_resource: illyrian
source_unit: aor harian night fighters, source_resource: harian
missing_factions: {'lugii'}
alt_source_unit: aor indian spearmen, alt_source_resource: indian
faction_exclude: ['ardiaei,', 'slave']


In [46]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [47]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("bithynian thureophoroi cavalry")

target_unit: aor bithynian thureophoroi cavalry
target_building: equestrian_2
target_level: 2
target_resource: bithynian
source_unit: aor babylonian horse archers, source_resource: babylonian
missing_factions: set()
alt_source_unit: aor caetrati cavalry, alt_source_resource: iberian
faction_exclude: ['bithynia,', 'slave']


In [48]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [50]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("mysian thureophoroi")

target_unit: aor mysian thureophoroi
target_building: barracks_2
target_level: 2
target_resource: mysian
source_unit: aor milesian hoplites, source_resource: milesian
missing_factions: {'miletus'}
alt_source_unit: aor nubian cavalry, alt_source_resource: ethiopian
faction_exclude: ['pergamon,', 'slave']


In [52]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)

In [53]:
target_unit, target_resource, target_level, source_unit, source_resource, missing_factions, alt_source_unit, \
    alt_source_resource, faction_exclude = get_details_for_aor_insertion("mysian cavalry")

target_unit: aor mysian cavalry
target_building: equestrian_3
target_level: 3
target_resource: mysian
source_unit: aor lyttian hetairoi, source_resource: lyttian
missing_factions: {'lyttos'}
alt_source_unit: aor numidian cavalry, alt_source_resource: numidian
faction_exclude: ['pergamon,', 'slave']


In [54]:
duplicate_unit_and_update(source_unit=source_unit, 
                          target_unit=target_unit, 
                          source_resource=source_resource, 
                          target_resource=target_resource,
                          alt_source_unit=alt_source_unit,
                          alt_source_resource=alt_source_resource,
                          building_level=target_level,
                          missing_factions=missing_factions,
                          faction_exclude=faction_exclude)